<a href="https://colab.research.google.com/github/OnMamiow/MyPortfolio.github.io/blob/main/Pull_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import requests
from datetime import datetime, timedelta

CLIENT_ID = "sh-2bb6e501-9c48-4a2e-8439-d56e7fb53524"
CLIENT_SECRET = "V0eaTYifKgdK7WeBnjaerHmhSr9ovHeM"

# 1. ขอ Access Token (ปรับปรุง)
def get_token():
    url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
    data = {
        "grant_type": "client_credentials",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET
    }

    try:
        response = requests.post(url, data=data)
        response.raise_for_status()  # เช็ค HTTP errors

        result = response.json()

        # Debug: แสดงผลลัพธ์
        print("Response:", result)

        if "access_token" not in result:
            print("Error: ไม่มี access_token ใน response")
            print("Full response:", result)
            return None

        return result["access_token"]

    except requests.exceptions.RequestException as e:
        print(f"HTTP Error: {e}")
        print(f"Response: {response.text if response else 'No response'}")
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None

# ทดสอบ
token = get_token()
if token:
    print("Token ได้แล้ว:", token[:50], "...")
else:
    print("ไม่สามารถขอ token ได้")


Response: {'access_token': 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJYVUh3VWZKaHVDVWo0X3k4ZF8xM0hxWXBYMFdwdDd2anhob2FPLUxzREZFIn0.eyJleHAiOjE3NjQ4NDEyMjIsImlhdCI6MTc2NDgzOTQyMiwianRpIjoiMTgzNzI4NzMtMzU4OS00MjAwLWFhZGItZDRkZThjOWQ1ODFmIiwiaXNzIjoiaHR0cHM6Ly9pZGVudGl0eS5kYXRhc3BhY2UuY29wZXJuaWN1cy5ldS9hdXRoL3JlYWxtcy9DRFNFIiwic3ViIjoiMjRmNWZlOGYtY2Q2Ni00ZDgyLTkwOTUtNzgyZGY0MTE5Mjc0IiwidHlwIjoiQmVhcmVyIiwiYXpwIjoic2gtMmJiNmU1MDEtOWM0OC00YTJlLTg0MzktZDU2ZTdmYjUzNTI0Iiwic2NvcGUiOiJlbWFpbCBwcm9maWxlIHVzZXItY29udGV4dCIsImVtYWlsX3ZlcmlmaWVkIjpmYWxzZSwiY2xpZW50SG9zdCI6IjEzNi4xMTcuMTU1LjIwOCIsIm9yZ2FuaXphdGlvbnMiOlsiZGVmYXVsdC1mNjE5MWU3YS0wMmM3LTQ1YjgtYjJjNS0xNjVhZjY4NTdkMzgiXSwidXNlcl9jb250ZXh0X2lkIjoiYWU3NGYwNDEtZWVmZC00N2Y1LWI1NGQtOWI1NWMzZTdkNDQzIiwiY29udGV4dF9yb2xlcyI6e30sImNvbnRleHRfZ3JvdXBzIjpbIi9hY2Nlc3NfZ3JvdXBzL3VzZXJfdHlwb2xvZ3kvY29wZXJuaWN1c19nZW5lcmFsLyIsIi9vcmdhbml6YXRpb25zL2RlZmF1bHQtZjYxOTFlN2EtMDJjNy00NWI4LWIyYzUtMTY1YWY2ODU3ZDM4LyJdLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJzZXJ2

In [2]:
from datetime import datetime, timedelta
import requests
import time

def get_satellite_image_swir(lat, lon, from_date, to_date, width=1024, height=1024, bbox_size=0.7):
    """
    Download SWIR composite satellite image from Sentinel-2.

    Args:
        lat: Latitude
        lon: Longitude
        from_date: Start date in dd/mm/yy format (e.g., "01/01/24")
        to_date: End date in dd/mm/yy format (e.g., "31/01/24")
        width: Image width in pixels
        height: Image height in pixels
        bbox_size: Size of bounding box
    """
    token = get_token()

    if not token:
        print("❌ Cannot get authentication token")
        print("Please update YOUR_USERNAME and YOUR_PASSWORD in get_token()")
        return None

    # แปลงวันที่จาก dd/mm/yy เป็น ISO 8601 format
    try:
        from_dt = datetime.strptime(from_date, "%d/%m/%y")
        to_dt = datetime.strptime(to_date, "%d/%m/%y")

        from_iso = from_dt.strftime("%Y-%m-%dT00:00:00Z")
        to_iso = to_dt.strftime("%Y-%m-%dT23:59:59Z")
    except ValueError as e:
        print(f"❌ รูปแบบวันที่ไม่ถูกต้อง: {e}")
        return None

    # Create bounding box
    bbox = [
        lon - bbox_size, lat - bbox_size,
        lon + bbox_size, lat + bbox_size
    ]

    url = "https://sh.dataspace.copernicus.eu/api/v1/process"

    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }

    # SWIR Composite evalscript
    payload = {
        "input": {
            "bounds": {
                "bbox": bbox,
                "properties": {"crs": "http://www.opengis.net/def/crs/EPSG/0/4326"}
            },
            "data": [{
                "type": "sentinel-2-l2a",
                "dataFilter": {
                    "timeRange": {
                        "from": from_iso,
                        "to": to_iso
                    },
                    "maxCloudCoverage": 30
                }
            }]
        },
        "output": {
            "width": width,
            "height": height,
            "responses": [{
                "identifier": "default",
                "format": {"type": "image/png"}
            }]
        },
        "evalscript": """
//VERSION=3
function setup() {
    return {
        input: ["B12", "B8A", "B04", "dataMask"],
        output: {
            bands: 4,
            sampleType: "AUTO"
        }
    };
}

function evaluatePixel(sample) {
    // SWIR Composite:
    // R = B12 (SWIR2) - Shows fire/heat/bare soil
    // G = B8A (NIR narrow) - Shows vegetation
    // B = B04 (Red) - Makes smoke appear cyan/blue

    return [
        2.5 * sample.B12,   // SWIR2 (enhanced for visibility)
        2.5 * sample.B8A,   // NIR
        2.5 * sample.B04,   // Red
        sample.dataMask     // Transparency
    ];
}
"""
    }

    print(f"🛰️  Requesting SWIR image for coordinates: {lat}, {lon}")
    print(f"   Searching from {from_date} to {to_date} (ISO: {from_iso} to {to_iso})")
    print(f"   Max cloud coverage: 30%")

    try:
        response = requests.post(url, headers=headers, json=payload, timeout=90)

        if response.status_code == 200:
            # ตรวจสอบว่าได้ข้อมูลภาพจริงหรือไม่
            if len(response.content) < 1000:
                print(f"⚠️  Warning: Image file is very small ({len(response.content)} bytes)")
                print(f"   This might indicate no satellite data available for this date")
                return None

            # สร้างชื่อไฟล์ที่มีวันที่ด้วย
            date_str = from_date.replace("/", "-")
            filename = f"satellite_SWIR_{lat}_{lon}_{date_str}.png"
            with open(filename, "wb") as f:
                f.write(response.content)
            print(f"✅ Successfully downloaded: {filename}")
            print(f"   File size: {len(response.content) / 1024:.2f} KB")
            return filename
        else:
            print(f"❌ Error: HTTP {response.status_code}")
            print(f"   Response: {response.text[:500]}")  # แสดงแค่ 500 ตัวอักษรแรก
            return None

    except requests.exceptions.Timeout:
        print("❌ Request timed out (90 seconds)")
        print("   Try: reducing image size, bbox_size, or check your internet connection")
        return None
    except requests.exceptions.RequestException as e:
        print(f"❌ Network error: {e}")
        return None
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return None


def download_images_every_15_days(lat, lon, start_date_str, end_date_str,
                                   width=1024, height=1024, bbox_size=0.7,
                                   date_range_days=3, max_retries=2):
    """
    ดาวน์โหลดภาพดาวเทียมทุกๆ 15 วัน ในช่วงเวลาที่กำหนด

    Args:
        lat: Latitude
        lon: Longitude
        start_date_str: วันที่เริ่มต้น (dd/mm/yy)
        end_date_str: วันที่สิ้นสุด (dd/mm/yy)
        width: ความกว้างภาพ (pixels)
        height: ความสูงภาพ (pixels)
        bbox_size: ขนาดพื้นที่ครอบคลุม
        date_range_days: ช่วงวันที่ในการค้นหา (±days) เพื่อเพิ่มโอกาสหาภาพ
        max_retries: จำนวนครั้งที่ลองใหม่เมื่อล้มเหลว
    """
    # แปลงวันที่จาก string เป็น datetime object
    try:
        start_date = datetime.strptime(start_date_str, "%d/%m/%y")
        end_date = datetime.strptime(end_date_str, "%d/%m/%y")
    except ValueError as e:
        print(f"❌ รูปแบบวันที่ไม่ถูกต้อง: {e}")
        print("   ใช้รูปแบบ: dd/mm/yy (เช่น 01/05/23)")
        return []

    current_date = start_date
    image_count = 0
    successful_downloads = []
    failed_dates = []

    print(f"\n{'='*70}")
    print(f"📅 เริ่มดาวน์โหลดภาพทุกๆ 15 วัน")
    print(f"   ช่วงเวลา: {start_date_str} ถึง {end_date_str}")
    print(f"   พิกัด: {lat}, {lon}")
    print(f"   ช่วงค้นหา: ±{date_range_days} วันจากวันที่ต้องการ")
    print(f"   ขนาดภาพ: {width}x{height} pixels")
    print(f"   พื้นที่ครอบคลุม: {bbox_size}° (ประมาณ {bbox_size * 111:.1f} km)")
    print(f"{'='*70}\n")

    while current_date <= end_date:
        image_count += 1

        # กำหนดช่วงวันที่ดาวน์โหลด (±date_range_days วัน)
        from_date_dt = current_date - timedelta(days=date_range_days)
        to_date_dt = current_date + timedelta(days=date_range_days)

        from_date = from_date_dt.strftime("%d/%m/%y")
        to_date = to_date_dt.strftime("%d/%m/%y")
        target_date = current_date.strftime("%d/%m/%y")

        print(f"\n{'─'*70}")
        print(f"[ภาพที่ {image_count}] วันที่เป้าหมาย: {target_date}")
        print(f"   ค้นหาในช่วง: {from_date} ถึง {to_date}")
        print(f"{'─'*70}")

        # ลองดาวน์โหลดพร้อม retry
        result = None
        for attempt in range(1, max_retries + 1):
            if attempt > 1:
                print(f"\n🔄 ลองครั้งที่ {attempt}/{max_retries}...")
                time.sleep(3)  # รอก่อนลองใหม่

            result = get_satellite_image_swir(
                lat=lat,
                lon=lon,
                from_date=from_date,
                to_date=to_date,
                width=width,
                height=height,
                bbox_size=bbox_size
            )

            if result:
                break

        if result:
            successful_downloads.append({
                'filename': result,
                'target_date': target_date,
                'search_range': f"{from_date} - {to_date}"
            })
            print(f"✅ สำเร็จ: {result}")
        else:
            failed_dates.append(target_date)
            print(f"❌ ไม่สามารถดาวน์โหลดภาพสำหรับวันที่ {target_date}")
            print(f"   สาเหตุที่เป็นไปได้:")
            print(f"   - ไม่มีภาพดาวเทียมในช่วงวันที่นี้")
            print(f"   - เมฆปกคลุมมากกว่า 30%")
            print(f"   - ดาวเทียมไม่ได้ผ่านพื้นที่นี้")

        # เพิ่ม 15 วัน
        current_date += timedelta(days=15)

        # หน่วงเวลาเล็กน้อยเพื่อไม่ให้ส่ง request เร็วเกินไป
        if current_date <= end_date:
            time.sleep(3)

    # สรุปผลลัพท์
    print(f"\n{'='*70}")
    print(f"📊 สรุปผลการดาวน์โหลด")
    print(f"{'='*70}")
    print(f"   ภาพทั้งหมด: {image_count} ภาพ")
    print(f"   ✅ ดาวน์โหลดสำเร็จ: {len(successful_downloads)} ภาพ")
    print(f"   ❌ ไม่สำเร็จ: {len(failed_dates)} ภาพ")

    if successful_downloads:
        print(f"\n📂 ไฟล์ที่ดาวน์โหลดสำเร็จ:")
        for i, item in enumerate(successful_downloads, 1):
            print(f"   {i}. {item['filename']}")
            print(f"      วันที่เป้าหมาย: {item['target_date']}")

    if failed_dates:
        print(f"\n⚠️  วันที่ดาวน์โหลดไม่สำเร็จ:")
        for date in failed_dates:
            print(f"   - {date}")
        print(f"\n💡 คำแนะนำ:")
        print(f"   - ลองเพิ่มค่า date_range_days (ปัจจุบัน: {date_range_days})")
        print(f"   - ลองเพิ่ม maxCloudCoverage เป็น 50-80%")
        print(f"   - เลือกวันที่อื่นที่อาจมีภาพดาวเทียม")

    print(f"{'='*70}\n")

    return successful_downloads


# Example usage
if __name__ == "__main__":
    # พิกัดสำหรับพื้นที่ Quebec, Canada
    lat = 53.015
    lon = -75.509

    # ดาวน์โหลดภาพทุกๆ 15 วัน ตั้งแต่ 01/05/23 ถึง 01/08/23
    results = download_images_every_15_days(
        lat=lat,
        lon=lon,
        start_date_str="01/05/23",  # 1 พฤษภาคม 2023
        end_date_str="01/08/23",    # 1 สิงหาคม 2023
        width=1024,
        height=1024,
        bbox_size=0.7,
        date_range_days=3,  # ค้นหา ±3 วัน จากวันที่เป้าหมาย
        max_retries=2       # ลองใหม่ 2 ครั้งถ้าล้มเหลว
    )

    print("\n📖 SWIR Composite Color Interpretation:")
    print("  🔴 Red/Pink areas: Heat, fire, bare soil, urban areas")
    print("  🟢 Green areas: Healthy vegetation")
    print("  🔵 Blue/Cyan areas: Smoke, water, or atmosphere")

    print("\n💾 Note: All images are saved in the current directory")
from datetime import datetime, timedelta
import requests
import time

def get_satellite_image_swir(lat, lon, from_date, to_date, width=1024, height=1024, bbox_size=0.7):
    """
    Download SWIR composite satellite image from Sentinel-2.

    Args:
        lat: Latitude
        lon: Longitude
        from_date: Start date in dd/mm/yy format (e.g., "01/01/24")
        to_date: End date in dd/mm/yy format (e.g., "31/01/24")
        width: Image width in pixels
        height: Image height in pixels
        bbox_size: Size of bounding box
    """
    token = get_token()

    if not token:
        print("❌ Cannot get authentication token")
        print("Please update YOUR_USERNAME and YOUR_PASSWORD in get_token()")
        return None

    # แปลงวันที่จาก dd/mm/yy เป็น ISO 8601 format
    try:
        from_dt = datetime.strptime(from_date, "%d/%m/%y")
        to_dt = datetime.strptime(to_date, "%d/%m/%y")

        from_iso = from_dt.strftime("%Y-%m-%dT00:00:00Z")
        to_iso = to_dt.strftime("%Y-%m-%dT23:59:59Z")
    except ValueError as e:
        print(f"❌ รูปแบบวันที่ไม่ถูกต้อง: {e}")
        return None

    # Create bounding box
    bbox = [
        lon - bbox_size, lat - bbox_size,
        lon + bbox_size, lat + bbox_size
    ]

    url = "https://sh.dataspace.copernicus.eu/api/v1/process"

    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }

    # SWIR Composite evalscript
    payload = {
        "input": {
            "bounds": {
                "bbox": bbox,
                "properties": {"crs": "http://www.opengis.net/def/crs/EPSG/0/4326"}
            },
            "data": [{
                "type": "sentinel-2-l2a",
                "dataFilter": {
                    "timeRange": {
                        "from": from_iso,
                        "to": to_iso
                    },
                    "maxCloudCoverage": 30
                }
            }]
        },
        "output": {
            "width": width,
            "height": height,
            "responses": [{
                "identifier": "default",
                "format": {"type": "image/png"}
            }]
        },
        "evalscript": """
//VERSION=3
function setup() {
    return {
        input: ["B12", "B8A", "B04", "dataMask"],
        output: {
            bands: 4,
            sampleType: "AUTO"
        }
    };
}

function evaluatePixel(sample) {
    // SWIR Composite:
    // R = B12 (SWIR2) - Shows fire/heat/bare soil
    // G = B8A (NIR narrow) - Shows vegetation
    // B = B04 (Red) - Makes smoke appear cyan/blue

    return [
        2.5 * sample.B12,   // SWIR2 (enhanced for visibility)
        2.5 * sample.B8A,   // NIR
        2.5 * sample.B04,   // Red
        sample.dataMask     // Transparency
    ];
}
"""
    }

    print(f"🛰️  Requesting SWIR image for coordinates: {lat}, {lon}")
    print(f"   Searching from {from_date} to {to_date} (ISO: {from_iso} to {to_iso})")
    print(f"   Max cloud coverage: 30%")

    try:
        response = requests.post(url, headers=headers, json=payload, timeout=90)

        if response.status_code == 200:
            # ตรวจสอบว่าได้ข้อมูลภาพจริงหรือไม่
            if len(response.content) < 1000:
                print(f"⚠️  Warning: Image file is very small ({len(response.content)} bytes)")
                print(f"   This might indicate no satellite data available for this date")
                return None

            # สร้างชื่อไฟล์ที่มีวันที่ด้วย
            date_str = from_date.replace("/", "-")
            filename = f"satellite_SWIR_{lat}_{lon}_{date_str}.png"
            with open(filename, "wb") as f:
                f.write(response.content)
            print(f"✅ Successfully downloaded: {filename}")
            print(f"   File size: {len(response.content) / 1024:.2f} KB")
            return filename
        else:
            print(f"❌ Error: HTTP {response.status_code}")
            print(f"   Response: {response.text[:500]}")  # แสดงแค่ 500 ตัวอักษรแรก
            return None

    except requests.exceptions.Timeout:
        print("❌ Request timed out (90 seconds)")
        print("   Try: reducing image size, bbox_size, or check your internet connection")
        return None
    except requests.exceptions.RequestException as e:
        print(f"❌ Network error: {e}")
        return None
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return None


def download_images_every_15_days(lat, lon, start_date_str, end_date_str,
                                   width=1024, height=1024, bbox_size=0.7,
                                   date_range_days=3, max_retries=2):
    """
    ดาวน์โหลดภาพดาวเทียมทุกๆ 15 วัน ในช่วงเวลาที่กำหนด

    Args:
        lat: Latitude
        lon: Longitude
        start_date_str: วันที่เริ่มต้น (dd/mm/yy)
        end_date_str: วันที่สิ้นสุด (dd/mm/yy)
        width: ความกว้างภาพ (pixels)
        height: ความสูงภาพ (pixels)
        bbox_size: ขนาดพื้นที่ครอบคลุม
        date_range_days: ช่วงวันที่ในการค้นหา (±days) เพื่อเพิ่มโอกาสหาภาพ
        max_retries: จำนวนครั้งที่ลองใหม่เมื่อล้มเหลว
    """
    # แปลงวันที่จาก string เป็น datetime object
    try:
        start_date = datetime.strptime(start_date_str, "%d/%m/%y")
        end_date = datetime.strptime(end_date_str, "%d/%m/%y")
    except ValueError as e:
        print(f"❌ รูปแบบวันที่ไม่ถูกต้อง: {e}")
        print("   ใช้รูปแบบ: dd/mm/yy (เช่น 01/05/23)")
        return []

    current_date = start_date
    image_count = 0
    successful_downloads = []
    failed_dates = []

    print(f"\n{'='*70}")
    print(f"📅 เริ่มดาวน์โหลดภาพทุกๆ 15 วัน")
    print(f"   ช่วงเวลา: {start_date_str} ถึง {end_date_str}")
    print(f"   พิกัด: {lat}, {lon}")
    print(f"   ช่วงค้นหา: ±{date_range_days} วันจากวันที่ต้องการ")
    print(f"   ขนาดภาพ: {width}x{height} pixels")
    print(f"   พื้นที่ครอบคลุม: {bbox_size}° (ประมาณ {bbox_size * 111:.1f} km)")
    print(f"{'='*70}\n")

    while current_date <= end_date:
        image_count += 1

        # กำหนดช่วงวันที่ดาวน์โหลด (±date_range_days วัน)
        from_date_dt = current_date - timedelta(days=date_range_days)
        to_date_dt = current_date + timedelta(days=date_range_days)

        from_date = from_date_dt.strftime("%d/%m/%y")
        to_date = to_date_dt.strftime("%d/%m/%y")
        target_date = current_date.strftime("%d/%m/%y")

        print(f"\n{'─'*70}")
        print(f"[ภาพที่ {image_count}] วันที่เป้าหมาย: {target_date}")
        print(f"   ค้นหาในช่วง: {from_date} ถึง {to_date}")
        print(f"{'─'*70}")

        # ลองดาวน์โหลดพร้อม retry
        result = None
        for attempt in range(1, max_retries + 1):
            if attempt > 1:
                print(f"\n🔄 ลองครั้งที่ {attempt}/{max_retries}...")
                time.sleep(3)  # รอก่อนลองใหม่

            result = get_satellite_image_swir(
                lat=lat,
                lon=lon,
                from_date=from_date,
                to_date=to_date,
                width=width,
                height=height,
                bbox_size=bbox_size
            )

            if result:
                break

        if result:
            successful_downloads.append({
                'filename': result,
                'target_date': target_date,
                'search_range': f"{from_date} - {to_date}"
            })
            print(f"✅ สำเร็จ: {result}")
        else:
            failed_dates.append(target_date)
            print(f"❌ ไม่สามารถดาวน์โหลดภาพสำหรับวันที่ {target_date}")
            print(f"   สาเหตุที่เป็นไปได้:")
            print(f"   - ไม่มีภาพดาวเทียมในช่วงวันที่นี้")
            print(f"   - เมฆปกคลุมมากกว่า 30%")
            print(f"   - ดาวเทียมไม่ได้ผ่านพื้นที่นี้")

        # เพิ่ม 15 วัน
        current_date += timedelta(days=15)

        # หน่วงเวลาเล็กน้อยเพื่อไม่ให้ส่ง request เร็วเกินไป
        if current_date <= end_date:
            time.sleep(3)

    # สรุปผลลัพท์
    print(f"\n{'='*70}")
    print(f"📊 สรุปผลการดาวน์โหลด")
    print(f"{'='*70}")
    print(f"   ภาพทั้งหมด: {image_count} ภาพ")
    print(f"   ✅ ดาวน์โหลดสำเร็จ: {len(successful_downloads)} ภาพ")
    print(f"   ❌ ไม่สำเร็จ: {len(failed_dates)} ภาพ")

    if successful_downloads:
        print(f"\n📂 ไฟล์ที่ดาวน์โหลดสำเร็จ:")
        for i, item in enumerate(successful_downloads, 1):
            print(f"   {i}. {item['filename']}")
            print(f"      วันที่เป้าหมาย: {item['target_date']}")

    if failed_dates:
        print(f"\n⚠️  วันที่ดาวน์โหลดไม่สำเร็จ:")
        for date in failed_dates:
            print(f"   - {date}")
        print(f"\n💡 คำแนะนำ:")
        print(f"   - ลองเพิ่มค่า date_range_days (ปัจจุบัน: {date_range_days})")
        print(f"   - ลองเพิ่ม maxCloudCoverage เป็น 50-80%")
        print(f"   - เลือกวันที่อื่นที่อาจมีภาพดาวเทียม")

    print(f"{'='*70}\n")

    return successful_downloads


# Example usage
if __name__ == "__main__":
    # พิกัดสำหรับพื้นที่ Quebec, Canada
    lat = 53.015
    lon = -75.509

    # ดาวน์โหลดภาพทุกๆ 15 วัน ตั้งแต่ 01/05/23 ถึง 01/08/23
    results = download_images_every_15_days(
        lat=lat,
        lon=lon,
        start_date_str="01/05/23",  # 1 พฤษภาคม 2023
        end_date_str="01/08/23",    # 1 สิงหาคม 2023
        width=1024,
        height=1024,
        bbox_size=0.7,
        date_range_days=3,  # ค้นหา ±3 วัน จากวันที่เป้าหมาย
        max_retries=2       # ลองใหม่ 2 ครั้งถ้าล้มเหลว
    )

    print("\n📖 SWIR Composite Color Interpretation:")
    print("  🔴 Red/Pink areas: Heat, fire, bare soil, urban areas")
    print("  🟢 Green areas: Healthy vegetation")
    print("  🔵 Blue/Cyan areas: Smoke, water, or atmosphere")

    print("\n💾 Note: All images are saved in the current directory")


📅 เริ่มดาวน์โหลดภาพทุกๆ 15 วัน
   ช่วงเวลา: 01/05/23 ถึง 01/08/23
   พิกัด: 53.015, -75.509
   ช่วงค้นหา: ±3 วันจากวันที่ต้องการ
   ขนาดภาพ: 1024x1024 pixels
   พื้นที่ครอบคลุม: 0.7° (ประมาณ 77.7 km)


──────────────────────────────────────────────────────────────────────
[ภาพที่ 1] วันที่เป้าหมาย: 01/05/23
   ค้นหาในช่วง: 28/04/23 ถึง 04/05/23
──────────────────────────────────────────────────────────────────────
Response: {'access_token': 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJYVUh3VWZKaHVDVWo0X3k4ZF8xM0hxWXBYMFdwdDd2anhob2FPLUxzREZFIn0.eyJleHAiOjE3NjQ4NDEyMjMsImlhdCI6MTc2NDgzOTQyMywianRpIjoiN2RmZmEwZGMtYmI3YS00ZTgwLTg2MjYtNDUwZTNkZmQzMmM5IiwiaXNzIjoiaHR0cHM6Ly9pZGVudGl0eS5kYXRhc3BhY2UuY29wZXJuaWN1cy5ldS9hdXRoL3JlYWxtcy9DRFNFIiwic3ViIjoiMjRmNWZlOGYtY2Q2Ni00ZDgyLTkwOTUtNzgyZGY0MTE5Mjc0IiwidHlwIjoiQmVhcmVyIiwiYXpwIjoic2gtMmJiNmU1MDEtOWM0OC00YTJlLTg0MzktZDU2ZTdmYjUzNTI0Iiwic2NvcGUiOiJlbWFpbCBwcm9maWxlIHVzZXItY29udGV4dCIsImVtYWlsX3ZlcmlmaWVkIjpmYWxzZSwiY2xpZW50SG9zdCI6IjEzN